In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from keras.utils import np_utils
from sklearn.datasets import load_files
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
train_files=load_files('train/')
train_imgpaths=train_files['filenames']
train_targets=np_utils.to_categorical(train_files['target'],len(train_files['target_names']))

In [3]:
def path_to_tensor(path):
    #load image
    img=image.load_img(path,target_size=[224,224])
    imgarray=image.img_to_array(img)
    imgarray=np.expand_dims(imgarray,axis=0)
    return imgarray

In [4]:
def paths_to_tensor(paths):
    listoftensors=[path_to_tensor(path) for path in paths]
    return np.vstack(listoftensors)

In [5]:
train_tensors=paths_to_tensor(train_imgpaths)


In [6]:
train_tensors=train_tensors/float(255)
train_tensors,valid_tensors,test_tensors=train_tensors[:-1700],train_tensors[-1700:-950],train_tensors[-950:]
train_targets,valid_targets,test_targets=train_targets[:-1700],train_targets[-1700:-950],train_targets[-950:]

In [7]:
from keras.layers import Dense,Convolution2D,MaxPooling2D,Dropout,Flatten
from keras.models import Sequential

In [8]:
snn=Sequential()
snn.add(Convolution2D(kernel_size=[3,3],filters=16,activation='relu',input_shape=[224,224,3]))
snn.add(MaxPooling2D(pool_size=[3,3]))
snn.add(Convolution2D(kernel_size=[3,3],filters=32,activation='relu'))
snn.add(MaxPooling2D(pool_size=[3,3]))
snn.add(Convolution2D(kernel_size=[3,3],filters=64,activation='relu'))
snn.add(MaxPooling2D(pool_size=[3,3]))
snn.add(Convolution2D(kernel_size=[3,3],filters=128,activation='relu'))
snn.add(MaxPooling2D(pool_size=[3,3]))
snn.add(Flatten())
snn.add(Dense(units=10,activation='relu'))
snn.add(Dropout(0.2))
snn.add(Dense(units=12,activation='softmax'))
snn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 5, 128)         73856     
__________

In [9]:
snn.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='rmsprop')

In [10]:
epochs=100
batch_size=30

from keras.callbacks import ModelCheckpoint

checkpointer=ModelCheckpoint(filepath='snnweights.best.hdf5',save_best_only=True,verbose=1)

snn.fit(train_tensors,train_targets,validation_data=[valid_tensors,valid_targets],batch_size=batch_size,
        epochs=epochs,shuffle=True,verbose=2,callbacks=[checkpointer])

Train on 3050 samples, validate on 750 samples
Epoch 1/100
Epoch 00000: val_loss improved from inf to 2.09112, saving model to snnweights.best.hdf5
11s - loss: 2.3048 - acc: 0.2256 - val_loss: 2.0911 - val_acc: 0.2813
Epoch 2/100
Epoch 00001: val_loss improved from 2.09112 to 1.93683, saving model to snnweights.best.hdf5
8s - loss: 2.0856 - acc: 0.2830 - val_loss: 1.9368 - val_acc: 0.3347
Epoch 3/100
Epoch 00002: val_loss improved from 1.93683 to 1.92864, saving model to snnweights.best.hdf5
8s - loss: 1.9509 - acc: 0.3213 - val_loss: 1.9286 - val_acc: 0.3560
Epoch 4/100
Epoch 00003: val_loss improved from 1.92864 to 1.72171, saving model to snnweights.best.hdf5
8s - loss: 1.8447 - acc: 0.3597 - val_loss: 1.7217 - val_acc: 0.4067
Epoch 5/100
Epoch 00004: val_loss improved from 1.72171 to 1.69669, saving model to snnweights.best.hdf5
8s - loss: 1.7472 - acc: 0.4056 - val_loss: 1.6967 - val_acc: 0.4200
Epoch 6/100
Epoch 00005: val_loss improved from 1.69669 to 1.58646, saving model to sn

In [13]:
snn.load_weights('snnweights.best.hdf5')

In [14]:
accuracy_test=snn.evaluate(test_tensors,test_targets)

928/950 [============================>.] - ETA: 0s

In [15]:
accuracy_test[1]*100

75.578947443711129